In [134]:
!wget -O "Train_stock_market.csv" "https://drive.usercontent.google.com/download?id=10QDNJURvLEBdTpc2r-ZuiHoCYzUsWBXU&export=download&confirm=t&uuid=980e8d75-95a1-466b-a95e-323a4f3d2224&at=APZUnTUOJyMDJkE7IC6Ec6Qi8alO:1699016695121"
!wget -O "model_4_parameters.json" "https://drive.google.com/uc?id=1hanZpmLnYFJgVcTNnQskLv2bnatJXKzH&export=download"

--2023-11-03 13:54:57--  https://drive.usercontent.google.com/download?id=10QDNJURvLEBdTpc2r-ZuiHoCYzUsWBXU&export=download&confirm=t&uuid=980e8d75-95a1-466b-a95e-323a4f3d2224&at=APZUnTUOJyMDJkE7IC6Ec6Qi8alO:1699016695121
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.197.132, 2607:f8b0:400e:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.197.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 761691 (744K) [application/octet-stream]
Saving to: ‘Train_stock_market.csv’

Train_stock_market. 100%[===================>] 743.84K  --.-KB/s    in 0.01s   

2023-11-03 13:54:58 (75.6 MB/s) - ‘Train_stock_market.csv’ saved [761691/761691]

--2023-11-03 13:54:58--  https://drive.google.com/uc?id=1hanZpmLnYFJgVcTNnQskLv2bnatJXKzH&export=download
Resolving drive.google.com (drive.google.com)... 74.125.197.113, 74.125.197.139, 74.125.197.101, ...
Connecting to drive.google.com (drive.google.com)|74.12

# CNN

In [135]:
# Convolutional layer sabar lagi mikir

# input = gambar
class Convolution :
  def __init__(self, filters, stride, mode, pool_filter_size, padding_size = 0) :
    self.padding_size = padding_size
    self.filters = filters
    self.stride = stride
    self.mode = mode
    self.pool_filter_size = pool_filter_size

  # Tahap convolution
  # input = np array height x width x channel
  # filter = array height x width x channel x nb_filter
  # padding_size = int
  # stride = int
  def convolution(self, input, padding_size, filters, stride) :
    # luaran = matriks size something < input_size
    input_height, input_width, input_channels = input.shape

    filter_height, filter_width, filter_channels, nb_filters = filters.shape

    # V = (W - F + 2P) / S + 1
    output_height = (input_height - filter_height + 2 * padding_size ) // stride + 1
    output_width = (input_width - filter_width + 2 * padding_size ) // stride + 1

    # output size = V * V * K
    # K = nb_filters
    output = np.zeros((output_height, output_width, nb_filters))

    # cooler version
    # slide per stride
    for i in range(0, input_height - filter_height + 1, stride):
        for j in range(0, input_width - filter_width + 1, stride):
            receptive_field = input[i:i+filter_height, j:j+filter_width, :]
            for f in range(nb_filters):
                output[i//stride, j//stride, f] = np.sum(receptive_field * filters[:, :, :, f])

    return output

  # Tahap detector
  def detector(self, input):
    # ni tinggal loop ke seluruh input data, trs relu 1 by 1
    return relu(input)

  # Tahap pooling
  # input = input (height x width x channel)
  # filter size = ukuran filter buat pooling
  # mode = max atau avg
  def pooling(self, input, filter_size, stride, mode) :
    # Ukuran input
    input_height, input_width, num_filter = input.shape

    # Ukuran output
    # formula sama tapi padding 0
    output_height = (input_height - filter_size) // stride + 1
    output_width = (input_width - filter_size) // stride + 1

    # Output pooling
    pooled_output = np.zeros((output_height, output_width, num_filter))

    # TODO: Optimize
    for i in range(0, output_height):
        for j in range(0, output_width):
            for f in range(num_filter):
                pooling_region = input[i*stride:i*stride+filter_size, j*stride:j*stride+filter_size, f]
                if mode == 'max':
                    max_value = pooling_region[0, 0]
                    for di in range(filter_size):
                        for dj in range(filter_size):
                            max_value = max(max_value, pooling_region[di, dj])
                    pooled_output[i, j, f] = max_value
                elif mode == 'average':
                    sum = 0
                    count = 0
                    for di in range(filter_size):
                        for dj in range(filter_size):
                            sum += pooling_region[di, dj]
                            count += 1
                    pooled_output[i, j, f] = sum / count


    return pooled_output

  # forward nglewatin semua tahap
  def forward(self, input) :
    conv_output = self.convolution(input, self.padding_size, self.filters, self.stride)
    detector_output = self.detector(conv_output)
    pooled_output = self.pooling(detector_output, self.pool_filter_size, self.stride, self.mode)
    return pooled_output

  def countParams(self):
      # Calculate the number of parameters
      filter_height, filter_width, filter_channels, nb_filters = self.filters.shape
      num_params = (filter_height * filter_width * filter_channels * nb_filters)
      return num_params

  def config(self):
    return {
        'filters': self.filters,
        'stride': self.stride,
        'mode': self.mode,
        'pool_filter_size': self.pool_filter_size,
        'padding_size': self.padding_size,
    }


In [136]:
class Flatten :
  def forward(self, input):
      # Flatten the input tensor
      flattened = input.reshape(-1)
      return flattened
  def config(self):
      return {}
  def countParams(self):
      return 0

# LESTEM

In [137]:
import numpy as np
import json

def linear(x):
  return x
def relu(x):
    return np.maximum(0, x)
def sigmoid(x):
  return 1 / (1 + np.exp(-x))
def tanh(x):
  return np.tanh(x)


In [138]:
tanh(0.1)

0.09966799462495582

In [139]:
def times(a : list[float], b: list[float]) -> float:
  #kali 1 1 elemen list dan menjumlahkannya
  if (len(a) != len(b)):
    raise Exception("A and B must be the same length")
  sum = 0.0
  for i in range(len(a)):
    sum += a[i] * b[i]
  return sum
def isEmpty(a : list) -> bool:
  return len(a) == 0

In [140]:
!pip install nptyping
from nptyping import NDArray, Shape, Float
#for typehints

In [141]:
class Cell:
  def __init__(self, num_unit: int, num_features: int, U : NDArray[Shape["4, *"], Float] = [], W : NDArray[Shape["4, *"], Float] = [], B: NDArray[Shape["4"], Float] = []):
    self.num_unit = num_unit
    self.num_features = num_features
    self.forget = 0.0
    self.state = 0.0
    self.input = 0.0
    self.output = 0.0
    self.candidate = 0.0
    self.hidden = 0.0
    if isEmpty(U):
      # self.U = np.zeros((4, num_features), dtype=np.dtype(float))
      self.U = np.random.rand(4,num_features)
    else:
      self.loadU(U)
    if isEmpty(W):
      # self.W = np.zeros((4, num_unit), dtype=np.dtype(float))
      self.W = np.random.rand(4,num_unit)
    else:
      self.loadW(W)
    if isEmpty(B):
      # self.B = np.zeros((4), dtype=np.dtype(float))
      self.B = np.random.rand(4)
    else:
      self.loadB(B)
    # 0 = i, 1 = f, 2 = c, 3 = O

  def loadU(self, U : NDArray[Shape["4, *"], Float]):
    m, n = U.shape
    if m != 4 or n != self.num_features:
      raise Exception("Size U must be 4 x num_features")
    self.U = U
  def loadW(self,  W : NDArray[Shape["4, *"], Float]):
    m, n = W.shape
    if m != 4 or n != self.num_unit:
      raise Exception("Size W must be 4 x num_unit")
    self.W = W
  def loadB(self, B: NDArray[Shape["4"], Float]):
    m = B.shape
    if m[0] != 4:
      raise Exception("Size B must be 4")
    self.B = B

  def forward(self, input, prevHidden : NDArray[Shape["4, *"], Float] = [], prevCell = 0.0):
    self.forget = self.calculateGates(input, prevHidden, 1)
    self.input = self.calculateGates(input, prevHidden, 0)
    self.output = self.calculateGates(input, prevHidden, 3)
    self.candidate = self.calculateGates(input, prevHidden, 2)
    self.state = self.forget * prevCell + self.input * self.candidate
    self.hidden = self.output * tanh(self.state)
    return self.hidden, self.state
  def calculateGates(self, input, prevHidden, mode: int):
    #prev hidden harusnya size num_unit, sama dengan W[mode]
    # 0 = i, 1 = f, 2 = c, 3 = O

    net = times(input,  self.U[mode])
    if not isEmpty(prevHidden):
      net += times(prevHidden , self.W[mode])
    net += self.B[mode]
    if (mode != 2):
      return sigmoid(net)
    else:
      return tanh(net)

  def countParams(self):
    return (self.U.size + self.W.size + self.B.size)

  def config(self):
      return {
          'num_unit': self.num_unit,
          'num_features': self.num_features,
          'U': self.U.tolist(),
          'W': self.W.tolist(),
          'B': self.B.tolist()
      }

In [142]:
class LSTM:
  def __init__(self, cellSize : int, inputSize : tuple [int, int], cells=None, cells_init=None):
    self.cellSize = cellSize
    self.inputSize = inputSize
    self.cells : list[Cell] = []
    self.outputs : list[float] = [0 for i in range(cellSize)]
    if cells_init:
      self.cells = cells_init
    elif cells :
      self.cells = [Cell(np.array(cell["num_unit"]), np.array(cell["num_features"]), np.array(cell["U"]), np.array(cell["W"]), np.array(cell["B"])) for cell in cells]
    else :
      self.cells = [Cell(self.cellSize, inputSize[1]) for _ in range(cellSize)]

  def forward(self,input):
    print(input.shape, self.inputSize)
    if (input.shape != self.inputSize):
      raise Exception( "Input shape must be equal to configurated inputSize!")
    hiddens = np.zeros((len(input), self.cellSize), dtype=np.dtype(float))
    states = np.zeros((len(input), self.cellSize), dtype=np.dtype(float))
    output = np.zeros((self.cellSize), dtype=np.dtype(float))
    for idxS, step in enumerate(input):
      for idxC, cell in enumerate(self.cells):
        if (idxS == 0):
          hidden, state = cell.forward(step)
        else:
          hidden, state = cell.forward(step, hiddens[idxS - 1, :], states[idxS - 1, idxC])
        hiddens[idxS, idxC] = hidden
        states[idxS, idxC] = state
    for idx, cell in enumerate(self.cells):
      output[idx] = cell.hidden
    return output

  def countParams(self):
      totalParams = 0
      for cell in self.cells:
          totalParams += cell.countParams()
      return totalParams

  def config(self):
      return {
          'cellSize': self.cellSize,
          'inputSize': self.inputSize,
          'cells': [cell.config() for cell in self.cells]
      }

# Dense

In [143]:
class Dense:
    def __init__(self, nb_units, activation_function='linear', weights=None, biases=None):
        self.nb_units = nb_units
        self.nb_input_unit = None
        self.activation_function = activation_function
        self.weights = weights
        self.biases = biases

    def forward(self, input):
        dense_output = np.dot(input, self.weights) + self.biases
        if self.activation_function == 'relu':
            activation_output = relu(dense_output)
        elif self.activation_function == 'sigmoid':
            activation_output = sigmoid(dense_output)
        elif self.activation_function == 'linear':
            activation_output = linear(dense_output)

        return activation_output

    def setNbInputUnit(self, nb_input_unit):
      self.nb_input_unit = nb_input_unit

    def setWeights(self, input):
      self.weights = np.random.randn(input, self.nb_units)

    def setBiases(self):
      self.biases = np.random.randn(self.nb_units)

    def countParams(self):
      return (self.nb_input_unit + 1) * self.nb_units

    def config(self):
      return {
          'nb_units': self.nb_units,
          'activation_function': self.activation_function,
          'weights': self.weights.tolist(),
          'biases': self.biases.tolist()
      }

# Sequential

In [144]:
class Sequential:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        if isinstance(layer, Dense):
          if isinstance(self.layers[-1], Flatten):
            layer.setNbInputUnit(0)
            layer.setWeights(1)
            layer.setBiases()
          elif isinstance(self.layers[-1], LSTM):
            layer.setNbInputUnit(self.layers[-1].cellSize)
            layer.setWeights(self.layers[-1].cellSize)
            layer.setBiases()
          else:
            layer.setNbInputUnit(self.layers[-1].nb_units)
            layer.setWeights(self.layers[-1].nb_units)
            layer.setBiases()
        self.layers.append(layer)

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def predict(self, x):
        predictions = []
        for input_data in x:
            output = self.forward(input_data)
            predictions.append(output)
        return np.array(predictions)

    def summary(self):
        totalParams = 0
        for i, layer in enumerate(self.layers):
            layerName = layer.__class__.__name__
            params = layer.countParams()
            totalParams += params
            print(f"Layer {i + 1}: {layerName} | Parameters: {params}")

        print(f"Total trainable parameters: {totalParams}")

    def countParams(self):
        totalParams = 0
        for layer in self.layers:
            totalParams += layer.countParams()
        return totalParams

    def save(self, filename):
        modelConfig = []
        for layer in self.layers:
            layerConfig = {
                'type': layer.__class__.__name__,
                'params': layer.config()
            }
            modelConfig.append(layerConfig)

        with open(filename, 'w') as jsonFile:
            json.dump(modelConfig, jsonFile)

    def load(self, filename):
        with open(filename, 'r') as jsonFile:
            modelConfig = json.load(jsonFile)

        loadedModel = Sequential()
        for layerConfig in modelConfig:
            layerType = layerConfig['type']
            layerParams = layerConfig['params']

            if layerType == 'LSTM':
                layer = LSTM(**layerParams)
            elif layerType == 'Dense':
                layer = Dense(**layerParams)
            elif layerType == 'Convolution':
                layer = Convolution(**layerParams)
            elif layerType == 'Flatten':
                layer = Flatten()
            else:
                raise ValueError(f"Unsupported layer type: {layerType}")

            loadedModel.add(layer)

        return loadedModel

    def loadAsisten(self, filename, window_size):
        with open(filename, 'r') as jsonFile:
            modelConfig = json.load(jsonFile)

        loadedModel = Sequential()
        for layerConfig in modelConfig:
            layerType = layerConfig['type']
            layerParams = layerConfig['params']

            if layerType == 'lstm':
                cells_init = [];
                num_cells = len(layerParams["W_i"][0])
                num_feat = len(layerParams["W_i"])
                # w = num_feat x num_unit
                # u = num_unit x num_unit
                # b = num_unit
                for i in range(num_cells):
                  w_i = np.transpose(layerParams["W_i"])[i]
                  w_f = np.transpose(layerParams["W_f"])[i]
                  w_c = np.transpose(layerParams["W_c"])[i]
                  w_o = np.transpose(layerParams["W_o"])[i]

                  u = np.array([w_i, w_f, w_c, w_o])
                  w = np.array([layerParams["U_i"][i], layerParams["U_f"][i], layerParams["U_c"][i], layerParams["U_o"][i]])
                  b = np.array([layerParams["b_i"][i], layerParams["b_f"][i], layerParams["b_c"][i], layerParams["b_o"][i]])

                  cells_init.append(Cell(num_cells, num_feat, u, w, b))
                layer = LSTM(num_cells, (window_size, num_feat), cells_init=cells_init)

            elif layerType == 'dense':
                nb_unit = len(layerParams["bias"])
                layer = Dense(nb_unit, 'linear', layerParams["kernel"], layerParams["bias"])
            else:
                raise ValueError(f"Unsupported layer type: {layerType}")

            loadedModel.add(layer)

        return loadedModel

In [145]:
def create_sequences(data, seq_length):
    sequences = []
    targets = []
    data_len = len(data)
    for i in range(data_len - seq_length):
        seq_end = i + seq_length
        seq_x = data[i:seq_end]
        seq_y = data[seq_end]
        sequences.append(seq_x)
        targets.append(seq_y)
    return np.array(sequences), np.array(targets)

# Load Data

In [146]:
import pandas as pd

df = pd.read_csv("/content/Train_stock_market.csv")
df

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,07-09-1984,5.25,5.500,7900,5.50,5.25,5.25
1,10-09-1984,5.25,5.250,600,5.50,5.25,5.25
2,11-09-1984,5.25,5.250,3500,5.50,5.25,5.25
3,12-09-1984,5.50,5.500,700,5.50,5.50,5.50
4,13-09-1984,5.00,5.500,1700,5.50,5.00,5.00
...,...,...,...,...,...,...,...
9640,06-12-2022,3.76,3.800,22400,3.99,3.81,3.81
9641,07-12-2022,3.68,3.750,18000,3.85,3.74,3.74
9642,08-12-2022,3.80,3.820,51600,4.00,3.85,3.85
9643,09-12-2022,3.85,3.930,7800,3.93,3.88,3.88


In [147]:
train = df.iloc[:, 1:6].values
train[-5:]

array([[3.75999999e+00, 3.79999995e+00, 2.24000000e+04, 3.99000001e+00,
        3.80999994e+00],
       [3.68000007e+00, 3.75000000e+00, 1.80000000e+04, 3.84999990e+00,
        3.74000001e+00],
       [3.79999995e+00, 3.81999993e+00, 5.16000000e+04, 4.00000000e+00,
        3.84999990e+00],
       [3.84999990e+00, 3.93000007e+00, 7.80000000e+03, 3.93000007e+00,
        3.88000011e+00],
       [3.84999990e+00, 3.87400007e+00, 1.14890000e+04, 3.88000011e+00,
        3.84999990e+00]])

# Eksperimen 1

In [148]:
window1 = 5
X_train, y_train = create_sequences(train, window1)

In [149]:
feature = 5
model1 = Sequential()
model1.add(LSTM(64, (window1, feature)))
model1.add(Dense(feature))
model1.summary()

Layer 1: LSTM | Parameters: 17920
Layer 2: Dense | Parameters: 325
Total trainable parameters: 18245


In [150]:
p1 = model1.predict([X_train[0]])
p1

(5, 5) (5, 5)


array([[ 3.51901281, -7.27631157, -8.04816255, -7.23382418, -0.15448723]])

In [151]:
y_train[0]

array([   5.  ,    5.  , 3300.  ,    5.5 ,    5.25])

In [152]:
diff = p1 - y_train[0]
print("RMSE: ", np.sqrt(np.mean(diff**2)))

RMSE:  1479.427382853852


# Eksperimen 2

In [153]:
window2 = 10
X_train, y_train = create_sequences(train, window2)

In [154]:
model2 = Sequential()
model2.add(LSTM(64, (window2, feature)))
model2.add(Dense(feature))
model2.summary()

Layer 1: LSTM | Parameters: 17920
Layer 2: Dense | Parameters: 325
Total trainable parameters: 18245


In [155]:
p2 = model2.predict([X_train[0]])
p2

(10, 5) (10, 5)


array([[ -3.29083968, -12.29767435,   7.02653853,  10.68183137,
         -7.67987589]])

In [156]:
y_train[0]

array([   5.25,    5.25, 3600.  ,    5.5 ,    5.5 ])

In [157]:
diff = p2 - y_train[0]
print("RMSE: ", np.sqrt(np.mean(diff**2)))

RMSE:  1606.8627646538691


# Eksperimen 3

In [158]:
window3 = 15
X_train, y_train = create_sequences(train, window3)

In [159]:
model3 = Sequential()
model3.add(LSTM(64, (window3, feature)))
model3.add(Dense(feature))
model3.summary()

Layer 1: LSTM | Parameters: 17920
Layer 2: Dense | Parameters: 325
Total trainable parameters: 18245


In [160]:
p3 = model3.predict([X_train[0]])
p3

(15, 5) (15, 5)


array([[-2.26401861, -8.62811557,  0.20187661, -1.22983838, -2.81627474]])

In [161]:
y_train[0]

array([6.00e+00, 6.25e+00, 1.78e+04, 6.50e+00, 6.25e+00])

In [162]:
diff = p3 - y_train[0]
print("RMSE: ", np.sqrt(np.mean(diff**2)))

RMSE:  7960.317139829661


# Eksperimen menggunakan Model Contoh

In [163]:
model = Sequential().loadAsisten("model_4_parameters.json", 15)
model.summary()

Layer 1: LSTM | Parameters: 17920
Layer 2: Dense | Parameters: 325
Total trainable parameters: 18245


In [164]:
pred = model.predict([X_train[0]])
pred

(15, 5) (15, 5)


<ipython-input-137-43ecad2c5645>:9: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


array([[-2.91575546,  2.21672772, -3.55727518, -0.48159397,  5.44156878]])

In [165]:
y_train[0]

array([6.00e+00, 6.25e+00, 1.78e+04, 6.50e+00, 6.25e+00])

In [166]:
diff = pred - y_train[0]
print("RMSE: ", np.sqrt(np.mean(diff**2)))

RMSE:  7961.994684812044
